In [6]:
# --- CELL 1: SETUP ---
import pandas as pd
import numpy as np
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))
from src.config import PROCESSED_DATA_PATH

# Define Paths (Hardcoded based on Phase 2 outputs)
FORECAST_PATH = "../data/processed/forecast_results.csv"
MASTER_OUTPUT_PATH = "../data/processed/powerbi_master_data.csv"

In [7]:
# --- CELL 2: LOAD DATA ---
# 1. Historical Data
print(f"Loading History from: {PROCESSED_DATA_PATH}")
df_history = pd.read_csv(PROCESSED_DATA_PATH, parse_dates=['Date'])
df_history['Type'] = 'Historical'

# 2. Forecast Data
print(f"Loading Forecast from: {FORECAST_PATH}")
df_forecast = pd.read_csv(FORECAST_PATH, parse_dates=['Date'])
df_forecast['Type'] = 'Forecast'

Loading History from: C:\VS Code\amdox-data-analytics\data\processed\processed_stock_data.csv
Loading Forecast from: ../data/processed/forecast_results.csv


In [8]:
# --- CELL 3: DATA TRANSFORMATION & MERGE ---
# Standardize columns for merging
# History has: Date, Close, etc.
# Forecast has: Date, Actual_Price, Predicted_Price, Lower_Conf, Upper_Conf

# Prepare History for Merge
df_hist_clean = pd.DataFrame({
    'Date': df_history['Date'],
    'Close_Price': df_history['Close'],
    'Type': 'Historical',
    'Lower_Bound': np.nan,
    'Upper_Bound': np.nan
})

# Prepare Forecast for Merge
df_fcast_clean = pd.DataFrame({
    'Date': df_forecast['Date'],
    'Close_Price': df_forecast['Predicted_Price'], # We plot prediction as the main line here
    'Type': 'Forecast',
    'Lower_Bound': df_forecast['Lower_Conf'],
    'Upper_Bound': df_forecast['Upper_Conf']
})

# Concatenate
df_master = pd.concat([df_hist_clean, df_fcast_clean], axis=0)
df_master.sort_values('Date', inplace=True)

In [9]:
# --- CELL 4: FEATURE ENGINEERING ---
# 1. Calculate 7-Day Moving Average (Trend)
df_master['MA_7'] = df_master['Close_Price'].rolling(window=7).mean()

# 2. Calculate Daily Volatility (Absolute % Change)
df_master['Volatility'] = df_master['Close_Price'].pct_change().abs() * 100

In [10]:
# --- CELL 5: EXPORT ---
df_master.to_csv(MASTER_OUTPUT_PATH, index=False)
print(f"SUCCESS: Master dataset saved to {MASTER_OUTPUT_PATH}")
print(f"Rows: {len(df_master)}")
print(df_master.tail())

SUCCESS: Master dataset saved to ../data/processed/powerbi_master_data.csv
Rows: 3321
           Date  Close_Price        Type  Lower_Bound  Upper_Bound       MA_7  \
299  2017-12-27        41.68    Forecast    20.827214    62.532786  54.031429   
3017 2017-12-28        69.82  Historical          NaN          NaN  58.051429   
300  2017-12-28        41.68    Forecast    20.792488    62.567512  53.780000   
3018 2017-12-29        69.85  Historical          NaN          NaN  57.804286   
301  2017-12-29        41.68    Forecast    20.757820    62.602180  57.804286   

      Volatility  
299    40.508136  
3017   67.514395  
300    40.303638  
3018   67.586372  
301    40.329277  
